#### Hier bereinige ich die Daten basierend auf den Problemen, die ich in der ersten Untersuchung gefunden habe.Ich versuche, jeden Schritt gut zu dokumentieren, damit ich später nachvollziehen kann, was ich gemacht habe.

#### Importiere benötigte Bibliotheken

In [5]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

#### Originaldaten laden

In [6]:
df = pd.read_csv('../data/raw/players_21.csv')
print(f"Originaldaten haben {df.shape[0]} Zeilen und {df.shape[1]} Spalten")

Originaldaten haben 18944 Zeilen und 106 Spalten


## 1. Größe und Gewicht bereinigen

In [7]:
def groesse_bereinigen(groesse_str):
    """Wandelt Größenangaben in Zentimeter um"""
    if isinstance(groesse_str, str):
        # Format ist z.B. "6'2" (6 Fuß 2 Inches)
        fuss_inches = groesse_str.split("'")
        fuss = int(fuss_inches[0])
        inches = int(fuss_inches[1].replace('"', '')) if len(fuss_inches) > 1 else 0
        return int((fuss * 30.48) + (inches * 2.54))  # Umrechnung in cm
    return groesse_str

In [8]:
def gewicht_bereinigen(gewicht_str):
    """Wandelt Gewichtsangaben in Kilogramm um"""
    if isinstance(gewicht_str, str):
        # Format ist z.B. "170lbs"
        gewicht_lbs = int(gewicht_str.replace('lbs', ''))
        return int(gewicht_lbs * 0.45359237)  # Umrechnung in kg
    return gewicht_str

In [11]:
print("Bereinige Größe und Gewicht...")
df['Groesse_cm'] = df['height_cm'].apply(groesse_bereinigen)
df['Gewicht_kg'] = df['weight_kg'].apply(gewicht_bereinigen)

Bereinige Größe und Gewicht...


## 2. Geldbeträge bereinigen

In [12]:
def geldbetrag_bereinigen(wert_str):
    """Wandelt Geldbeträge in Zahlen um"""
    if isinstance(wert_str, str):
        wert_str = wert_str.replace('€', '')
        multiplikator = 1
        
        if 'M' in wert_str:  # Millionen
            multiplikator = 1000000
            wert_str = wert_str.replace('M', '')
        elif 'K' in wert_str:  # Tausend
            multiplikator = 1000
            wert_str = wert_str.replace('K', '')
            
        return float(wert_str) * multiplikator
    return wert_str

In [17]:
print("Bereinige Geldbeträge...")
df['Marktwert_EUR'] = df['value_eur'].apply(geldbetrag_bereinigen)
df['Gehalt_EUR'] = df['wage_eur'].apply(geldbetrag_bereinigen)
df['Ausstiegsklausel_EUR'] = df['release_clause_eur'].apply(geldbetrag_bereinigen)

Bereinige Geldbeträge...


## 3. Team und Vertrag aufteilen

In [ ]:
def team_vertrag_aufteilen(team_vertrag):
    """Teilt Team- und Vertragsinformationen auf"""
    if pd.isna(team_vertrag):
        return pd.Series({'club_name': None, 'joined': None, 'contract_valid_until': None})
    
    teile = team_vertrag.split(' ')
    team = ' '.join(teile[:-2])
    vertragsjahre = teile[-1]
    
    try:
        start_jahr, end_jahr = vertragsjahre.split('~')
        return pd.Series({
            'club_name': team.strip(),
            'joined': start_jahr.strip(),
            'contract_valid_until': end_jahr.strip()
        })
    except:
        return pd.Series({'club_name': team.strip(), 'joined': None, 'contract_valid_until': None})


In [57]:
print("Teile Team und Vertragsinformationen auf...")
df[['Team', 'Vertrag_Start', 'Vertrag_Ende']] = df['Team & Contract'].apply(team_vertrag_aufteilen)

Teile Team und Vertragsinformationen auf...


KeyError: 'Team & Contract'

## 4. Hits-Spalte bereinigen

In [35]:
# print("Bereinige die Hits-Spalte...")
# df['Hits_Bereinigt'] = df['hits'].str.replace('\n', ' ').str.strip()

## 5. Fehlende Werte behandeln

In [36]:
print("Behandle fehlende Werte...")
numerische_spalten = df.select_dtypes(include=[np.number]).columns
df[numerische_spalten] = df[numerische_spalten].fillna(0)

Behandle fehlende Werte...


## 6. Datumsangaben vereinheitlichen

In [37]:
def beitrittsdatum_bereinigen(datum_str):
    """Vereinheitlicht Datumsangaben"""
    if pd.isna(datum_str):
        return None
    try:
        return pd.to_datetime(datum_str).strftime('%Y-%m-%d')
    except:
        return None

In [39]:
print("Vereinheitliche Datumsangaben...")
df['Beitrittsdatum'] = df['joined'].apply(beitrittsdatum_bereinigen)

Vereinheitliche Datumsangaben...


## 7. Abschließende Bereinigung

In [49]:
print("Führe abschließende Bereinigung durch...")
zu_loeschende_spalten = ['height_cm', 'weight_kg', 'value_eur', 'wage_eur', 'release_clause_eur']
df_bereinigt = df.drop(columns=zu_loeschende_spalten)

Führe abschließende Bereinigung durch...


#### Bereinigte Daten speichern

In [50]:
print("Speichere bereinigte Daten...")
df_bereinigt.to_csv('../data/processed/fifa21_bereinigt.csv', index=False)
print("Bereinigung abgeschlossen!")

Speichere bereinigte Daten...
Bereinigung abgeschlossen!


#### Beispiel der bereinigten Daten zeigen

In [51]:
print("\nBeispiel der bereinigten Daten:")
display(df_bereinigt.head())


Beispiel der bereinigten Daten:


,sofifa_id,player_url,short_name,long_name,age,dob,nationality,club_name,league_name,league_rank,...,lcb,cb,rcb,rb,Groesse_cm,Gewicht_kg,Marktwert_EUR,Gehalt_EUR,Ausstiegsklausel_EUR,Beitrittsdatum
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,33,1987-06-24,Argentina,FC Barcelona,Spain Primera Division,1.0,...,52+3,52+3,52+3,62+3,170,72,67500000,560000,138400000.0,2004-07-01
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,35,1985-02-05,Portugal,Juventus,Italian Serie A,1.0,...,54+3,54+3,54+3,61+3,187,83,46000000,220000,75900000.0,2018-07-10
2,200389,https://sofifa.com/player/200389/jan-oblak/210002,J. Oblak,Jan Oblak,27,1993-01-07,Slovenia,Atlético Madrid,Spain Primera Division,1.0,...,33+3,33+3,33+3,32+3,188,87,75000000,125000,159400000.0,2014-07-16
3,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,31,1988-08-21,Poland,FC Bayern München,German 1. Bundesliga,1.0,...,60+3,60+3,60+3,61+3,184,80,80000000,240000,132000000.0,2014-07-01
4,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,28,1992-02-05,Brazil,Paris Saint-Germain,French Ligue 1,1.0,...,49+3,49+3,49+3,62+3,175,68,90000000,270000,166500000.0,2017-08-03


### Zusammenfassung der Änderungen

In [53]:
print("\nZusammenfassung der Bereinigung:")
print(f"Ursprüngliche Spalten: {len(df.columns)}")
print(f"Bereinigte Spalten: {len(df_bereinigt.columns)}")
print("\nNeue Spalten:")
neue_spalten = ['Groesse_cm', 'Gewicht_kg', 'Marktwert_EUR', 'Gehalt_EUR', 'Ausstiegsklausel_EUR', 
               'Team', 'Vertrag_Start', 'Vertrag_Ende', 'Hits_Bereinigt', 'Beitrittsdatum']
for spalte in neue_spalten:
    print(f"- {spalte}")

print("\nIch habe die Daten jetzt so aufbereitet, dass man den shit besser analysieren kann!")


Zusammenfassung der Bereinigung:
Ursprüngliche Spalten: 112
Bereinigte Spalten: 107

Neue Spalten:
- Groesse_cm
- Gewicht_kg
- Marktwert_EUR
- Gehalt_EUR
- Ausstiegsklausel_EUR
- Team
- Vertrag_Start
- Vertrag_Ende
- Hits_Bereinigt
- Beitrittsdatum

Ich habe die Daten jetzt so aufbereitet, dass man den shit besser analysieren kann!
